# ⚽ Análise Tática de Futebol com Visão Computacional

**Alunos:** Andreia, Claudio Filho, Kelwin Menezes, Felipe Pereira,Laura Menezes, Lucas Ungarelli, Pedro Araújo

**Disciplina:** Processamento Inteligente de Imagens Aeroespaciais  

**Universidade:** IDP

**Ano:** 2025

---

## 1. O Desafio: A "Estória do Cliente"

O ponto de partida foi a necessidade de um utilizador leigo em futebol que, ao ouvir os comentadores a falar sobre "esquemas táticos", "organização coletiva" e "posse de bola", não conseguia visualizar esses conceitos no jogo. O objetivo deste projeto é ser a ferramenta que preenche essa lacuna, traduzindo a linguagem do futebol em *insights* visuais claros, credíveis e compreensíveis.

## 2. Instalação das Dependências

O comando abaixo instala todas as bibliotecas Python necessárias para executar o projeto.

In [ ]:
!pip install opencv-python ultralytics norfair numpy pillow scipy

## 3. Estrutura do Projeto e Código-Fonte

O projeto foi organizado numa estrutura modular para garantir a clareza e a reutilização do código. As células a seguir contêm o código-fonte completo, dividido pelos seus módulos originais.

### 3.1. Módulo de Configuração

Define os filtros de cores HSV para a classificação das equipas e do árbitro.

In [ ]:
# config/filtros_cores.py
preto = {"name": "preto", "lower_hsv": (0, 0, 0), "upper_hsv": (179, 255, 45)}
branco = {"name": "branco", "lower_hsv": (0, 0, 180), "upper_hsv": (179, 30, 255)}
verde = {"name": "verde", "lower_hsv": (40, 40, 40), "upper_hsv": (80, 255, 255)}
rosa = {"name": "rosa", "lower_hsv": (145, 60, 100), "upper_hsv": (175, 255, 255)}
amarelo = {"name": "amarelo", "lower_hsv": (22, 93, 0), "upper_hsv": (45, 255, 255)}

filtro_arbitro = {"name": "Referee", "colors": [amarelo, preto]}
filtro_inter_miami = {"name": "Inter Miami", "colors": [rosa]}
filtro_palmeiras = {"name": "Palmeiras", "colors": [branco, verde]}

# Ordem de prioridade: Árbitro primeiro, para evitar confusão com outras cores
filtros = [filtro_arbitro, filtro_inter_miami, filtro_palmeiras]

### 3.2. Módulo de Inferência (`inference`)

Contém as classes responsáveis pela deteção de objetos (usando YOLOv8) e pela classificação de equipas (usando filtros HSV e inércia).

In [ ]:
# inference/base_detector.py
from abc import ABC, abstractmethod
import pandas as pd
class BaseDetector(ABC):
    @abstractmethod
    def predict(self, input_image): pass

# inference/base_classifier.py
class BaseClassifier(ABC):
    @abstractmethod
    def predict(self, input_image): pass

# inference/box.py
class Box:
    def __init__(self, top_left, bottom_right, img):
        self.top_left = (int(top_left[0]), int(top_left[1]))
        self.bottom_right = (int(bottom_right[0]), int(bottom_right[1]))
        self.img = self.cut(img.copy())
    def cut(self, img):
        return img[self.top_left[1]:self.bottom_right[1], self.top_left[0]:self.bottom_right[0]]
        
# inference/converter.py
import norfair
import numpy as np
class Converter:
    @staticmethod
    def DataFrame_to_Detections(df: pd.DataFrame):
        detections = []
        for _, row in df.iterrows():
            box = np.array([[row["xmin"], row["ymin"]], [row["xmax"], row["ymax"]]])
            data = {"name": row["name"], "p": row.get("confidence", 0)}
            detections.append(norfair.Detection(points=box, data=data))
        return detections
    @staticmethod
    def TrackedObjects_to_Detections(tracked_objects):
        detections = []
        for obj in tracked_objects:
            if obj.live_points.any():
                detection = obj.last_detection
                detection.data["id"] = int(obj.id)
                detections.append(detection)
        return detections

# inference/detector.py
from typing import List
from ultralytics import YOLO
class Detector(BaseDetector):
    def __init__(self, model_path: str = "yolov8x.pt"):
        try:
            self.model = YOLO(model_path)
            self.device = self.model.device
        except Exception as e:
            raise e

    def predict(self, input_image: List[np.ndarray]) -> pd.DataFrame:
        results = self.model(input_image, verbose=False)
        all_preds_df = []
        for result in results:
            if result.boxes:
                preds_df = pd.DataFrame(result.boxes.data.cpu().numpy(), columns=['xmin', 'ymin', 'xmax', 'ymax', 'confidence', 'class'])
                names = result.names
                preds_df['name'] = preds_df['class'].apply(lambda x: names[int(x)])
                all_preds_df.append(preds_df)
        if not all_preds_df: return pd.DataFrame()
        return pd.concat(all_preds_df, ignore_index=True)

# inference/hsv_classifier.py
import cv2
class HSVClassifier(BaseClassifier):
    def __init__(self, filters):
        self.filters = filters
    def predict(self, input_images):
        return [self._predict_img(img) for img in input_images]
    def _predict_img(self, img):
        if img is None or img.size == 0: return "unknown"
        max_pixels = -1
        predicted_team = "unknown"
        for team_filter in self.filters:
            pixel_count = 0
            for color in team_filter["colors"]:
                pixel_count += self._count_color_pixels(img, color)
            if pixel_count > max_pixels:
                max_pixels = pixel_count
                predicted_team = team_filter["name"]
        return predicted_team
    def _count_color_pixels(self, img, color_filter):
        img_hsv = cv2.cvtColor(self._crop_jersey(img), cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(img_hsv, color_filter["lower_hsv"], color_filter["upper_hsv"])
        return cv2.countNonZero(mask)
    def _crop_jersey(self, img):
        h, w, _ = img.shape
        return img[int(h * 0.15):int(h * 0.6), int(w * 0.1):int(w * 0.9)]

# inference/inertia_classifier.py
class InertiaClassifier:
    def __init__(self, classifier, inertia=25):
        self.classifier = classifier
        self.inertia = inertia
        self.classifications_per_id = {}
    def predict_from_detections(self, detections, img):
        if not detections: return []
        imgs_to_classify = [self._get_img_from_detection(det, img) for det in detections]
        predictions = self.classifier.predict(imgs_to_classify)
        for det, pred in zip(detections, predictions):
            det_id = det.data.get("id")
            if det_id not in self.classifications_per_id:
                self.classifications_per_id[det_id] = []
            self.classifications_per_id[det_id].append(pred)
            if len(self.classifications_per_id[det_id]) > self.inertia:
                self.classifications_per_id[det_id].pop(0)
            det.data["classification"] = max(set(self.classifications_per_id[det_id]), key=self.classifications_per_id[det_id].count)
        return detections
    def _get_img_from_detection(self, det, img):
        p1, p2 = det.points.astype(int)
        return img[p1[1]:p2[1], p1[0]:p2[0]]

### 3.3. Módulo de Lógica de Jogo (`soccer`)

Contém as classes que representam a lógica e os elementos de uma partida de futebol: `Time`, `Jogador`, `Bola`, `Partida`, etc.

In [ ]:
# soccer/time.py
class Time:
    def __init__(self, nome, abreviacao, cor, cor_placar=None, cor_texto=(255, 255, 255)):
        self.nome = nome
        self.abreviacao = abreviacao
        self.cor = cor
        self.cor_texto = cor_texto
        self.cor_placar = cor_placar if cor_placar is not None else cor
        self.posse_de_bola_frames = 0
        self.passes = []

    def obter_percentual_posse(self, duracao_total_frames: int) -> float:
        if duracao_total_frames == 0: return 0
        return round(self.posse_de_bola_frames / duracao_total_frames, 2)

    def obter_tempo_posse_formatado(self, fps: int) -> str:
        segundos = round(self.posse_de_bola_frames / fps) if fps > 0 else 0
        minutos = int(segundos // 60)
        segundos = int(segundos % 60)
        return f"{minutos:02d}:{segundos:02d}"

    def __str__(self): return self.nome
    def __eq__(self, other): return isinstance(other, Time) and self.nome == other.nome

# soccer/desenho.py
import PIL
from PIL import ImageDraw, ImageFont
class Desenho:
    @staticmethod
    def obter_fonte(tamanho=18):
        try:
            return PIL.ImageFont.truetype("DejaVuSans.ttf", size=tamanho)
        except IOError:
            return PIL.ImageFont.load_default()

    @staticmethod
    def desenhar_deteccao(detection, img, cor):
        desenhador = ImageDraw.Draw(img)
        desenhador.rounded_rectangle([tuple(p) for p in detection.points], radius=5, outline=cor, width=3)
        return img

    @staticmethod
    def desenhar_ponteiro(detection, img, cor):
        x1, y1 = detection.points[0]; x2, _ = detection.points[1]
        centro_x = (x1 + x2) / 2
        desenhador = ImageDraw.Draw(img)
        pontos = [(centro_x, y1 - 8), (centro_x - 10, y1 - 25), (centro_x + 10, y1 - 25)]
        desenhador.polygon(pontos, fill=cor, outline="black", width=1)
        return img

    @staticmethod
    def desenhar_painel_posse(img, partida):
        desenhador = ImageDraw.Draw(img, "RGBA")
        painel_rect = (10, 10, 310, 90)
        desenhador.rounded_rectangle(painel_rect, radius=10, fill=(0, 0, 0, 180))

        fonte_titulo = Desenho.obter_fonte(16); fonte_time = Desenho.obter_fonte(20)
        
        desenhador.text((25, 18), "POSSE DE BOLA", fill="white", font=fonte_titulo)
        
        casa, fora = partida.time_casa, partida.time_visitante
        
        if not partida.time_com_posse:
            fonte_sem_posse = Desenho.obter_fonte(22)
            desenhador.text((90, 45), "SEM POSSE", fill=(200, 200, 200), font=fonte_sem_posse)
        else:
            posse_casa = casa.obter_percentual_posse(partida.duracao_total_frames) * 100
            posse_fora = fora.obter_percentual_posse(partida.duracao_total_frames) * 100

            desenhador.text((25, 45), f"{casa.abreviacao}", fill=casa.cor, font=fonte_time)
            desenhador.text((90, 45), f"{posse_casa:.1f}%", fill="white", font=fonte_time)
            
            desenhador.text((170, 45), f"{fora.abreviacao}", fill=fora.cor, font=fonte_time)
            desenhador.text((235, 45), f"{posse_fora:.1f}%", fill="white", font=fonte_time)
            
        return img

# soccer/jogador.py
class Jogador:
    def __init__(self, detection):
        self.detection = detection
        self.time = detection.data.get("team")

    @property
    def centro(self) -> np.ndarray:
        if not self.detection: return None
        x1, y1 = self.detection.points[0]; x2, y2 = self.detection.points[1]
        return np.array([(x1 + x2) / 2, (y1 + y2) / 2])

    def distancia_para_bola(self, bola) -> float:
        if self.centro is None or bola.centro is None: return float('inf')
        return np.linalg.norm(self.centro - bola.centro)

    def desenhar(self, frame, **kwargs):
        if self.detection:
            cor = self.time.cor if self.time else (128, 128, 128)
            return Desenho.desenhar_deteccao(self.detection, frame, cor=cor)
        return frame

    def desenhar_ponteiro(self, frame):
        if self.detection and self.time:
            return Desenho.desenhar_ponteiro(self.detection, frame, self.time.cor)
        return frame

    @staticmethod
    def criar_lista_de_deteccoes(detections, times):
        jogadores = []
        for det in detections:
            if "classification" in det.data:
                nome_time = det.data["classification"]
                det.data["team"] = next((t for t in times if t.nome == nome_time), None)
            jogadores.append(Jogador(det))
        return jogadores
        
# soccer/bola.py
class Bola:
    def __init__(self, detection):
        self.detection = detection
        self.cor = (255, 255, 255)
        self.pontos_rastro = []
        self.max_comprimento_rastro = 20
        self.rastro_habilitado = True

    def definir_cor(self, partida):
        if partida and partida.time_com_posse:
            self.cor = partida.time_com_posse.cor
        else:
            self.cor = (255, 255, 255)
        
        if self.detection:
            self.detection.data["color"] = self.cor

    @property
    def centro(self) -> tuple:
        if self.detection is None: return None
        x1, y1 = self.detection.points[0]; x2, y2 = self.detection.points[1]
        return (np.round_((x1 + x2) / 2), np.round_((y1 + y2) / 2))

    def atualizar_rastro(self):
        if self.detection and self.rastro_habilitado and self.centro is not None:
            self.pontos_rastro.append(self.centro)
            if len(self.pontos_rastro) > self.max_comprimento_rastro:
                self.pontos_rastro.pop(0)

    def desenhar_rastro(self, frame: PIL.Image.Image) -> PIL.Image.Image:
        if len(self.pontos_rastro) < 2: return frame
        desenhador = ImageDraw.Draw(frame, "RGBA")
        for i in range(1, len(self.pontos_rastro)):
            alfa = int(255 * (i / len(self.pontos_rastro)))
            largura = max(1, int(8 * (i / len(self.pontos_rastro))))
            cor_com_alfa = self.cor + (alfa,)
            ponto_inicial = tuple(map(int, self.pontos_rastro[i - 1]))
            ponto_final = tuple(map(int, self.pontos_rastro[i]))
            desenhador.line([ponto_inicial, ponto_final], fill=cor_com_alfa, width=largura)
        return frame

    def desenhar(self, frame: PIL.Image.Image) -> PIL.Image.Image:
        self.atualizar_rastro()
        if self.rastro_habilitado:
            frame = self.desenhar_rastro(frame)
        if self.detection:
            frame = Desenho.desenhar_deteccao(self.detection, frame, self.cor)
        return frame

# soccer/visualizacao_tatica.py
from scipy.spatial import ConvexHull
import math
class VisualizacaoTatica:
    def __init__(self):
        self.alfa_poligono = 45
        self.alfa_linha = 100
        self.max_conexoes_por_jogador = 2
        self.distancia_max_conexao = 350

    def obter_jogadores_por_time(self, jogadores, time):
        return [j for j in jogadores if j.time == time and j.detection]

    def obter_posicoes_jogadores(self, jogadores):
        return [tuple(j.centro) for j in jogadores if j.centro is not None]

    def desenhar_linhas_conexao(self, img, posicoes, cor):
        if len(posicoes) < 2: return img
        desenhador = ImageDraw.Draw(img, "RGBA")
        cor_com_alfa = cor + (self.alfa_linha,)
        for i, pos1 in enumerate(posicoes):
            distancias = sorted([(math.dist(pos1, pos2), pos2) for j, pos2 in enumerate(posicoes) if i != j and math.dist(pos1, pos2) < self.distancia_max_conexao])
            for k in range(min(self.max_conexoes_por_jogador, len(distancias))):
                desenhador.line([pos1, distancias[k][1]], fill=cor_com_alfa, width=2)
        return img

    def desenhar_poligono_formacao(self, img, posicoes, cor):
        if len(posicoes) < 3: return img
        try:
            pontos_np = np.array(posicoes)
            casco = ConvexHull(pontos_np)
            pontos_casco = [tuple(p) for p in pontos_np[casco.vertices]]
            desenhador = ImageDraw.Draw(img, "RGBA")
            desenhador.polygon(pontos_casco, fill=cor+(self.alfa_poligono,), outline=cor+(self.alfa_poligono+40,), width=2)
            
            centroide = np.mean(pontos_np, axis=0)
            cx, cy = int(centroide[0]), int(centroide[1])
            raio = 5
            desenhador.ellipse((cx-raio, cy-raio, cx+raio, cy+raio), fill=cor)
        except Exception: pass
        return img

    def desenhar_analise_tatica(self, img, jogadores, times, args):
        for time in times:
            jogadores_time = self.obter_jogadores_por_time(jogadores, time)
            posicoes = self.obter_posicoes_jogadores(jogadores_time)
            if not posicoes: continue
            if args.poligonos_formacao:
                img = self.desenhar_poligono_formacao(img, posicoes, time.cor)
            if args.linhas_formacao:
                img = self.desenhar_linhas_conexao(img, posicoes, time.cor)
        return img
        
# soccer/partida.py
class Partida:
    def __init__(self, time_casa: Time, time_visitante: Time, fps: int):
        self.times = [time_casa, time_visitante]
        self.time_casa = time_casa
        self.time_visitante = time_visitante
        self.fps = fps if fps > 0 else 30.0
        self.duracao_total_frames = 0
        self.bola = None
        self.jogador_mais_proximo = None
        self.time_com_posse = None
        self.limiar_distancia_posse = 120.0
        self.visualizador_tatico = VisualizacaoTatica()

    def atualizar(self, jogadores: list, bola: Bola):
        self.duracao_total_frames += 1
        self.bola = bola
        
        jogador_com_posse_neste_frame = None
        
        if jogadores and bola.detection:
            jogadores_com_time = [j for j in jogadores if j.time]
            if jogadores_com_time:
                jogador_candidato = min(jogadores_com_time, key=lambda j: j.distancia_para_bola(bola))
                distancia = jogador_candidato.distancia_para_bola(bola)

                if distancia < self.limiar_distancia_posse:
                    jogador_com_posse_neste_frame = jogador_candidato
                    self.time_com_posse = jogador_candidato.time
        
        self.jogador_mais_proximo = jogador_com_posse_neste_frame
        
        if self.time_com_posse:
            self.time_com_posse.posse_de_bola_frames += 1

    def desenhar_elementos(self, frame, jogadores, args):
        if self.bola and args.rastro_bola:
            self.bola.definir_cor(self)
            frame = self.bola.desenhar(frame)
        
        if args.linhas_formacao or args.poligonos_formacao:
            frame = self.visualizador_tatico.desenhar_analise_tatica(frame, jogadores, self.times, args)
        
        if self.jogador_mais_proximo and args.posse:
            frame = self.jogador_mais_proximo.desenhar_ponteiro(frame)

        if args.posse:
            frame = Desenho.desenhar_painel_posse(frame, self)
            
        return frame

### 3.4. Módulo de Utilitários (`utils`)

Funções auxiliares para o processamento principal, como obter deteções e atualizar o estimador de movimento da câmara.

In [ ]:
# utils/funcoes_execucao.py
from typing import List
from norfair import Detection
from norfair.camera_motion import MotionEstimator

def obter_deteccoes_bola(detector_bola: Detector, frame: np.ndarray, usar_bola_esportiva: bool = False) -> List[Detection]:
    df_bola = detector_bola.predict(frame)
    if usar_bola_esportiva and not df_bola.empty:
        if 'name' in df_bola.columns:
            df_bola = df_bola[df_bola["name"] == "sports ball"]
        elif 'class' in df_bola.columns:
            df_bola = df_bola[df_bola["class"] == 32]
    return Converter.DataFrame_to_Detections(df_bola[df_bola["confidence"] > 0.15])

def obter_deteccoes_jogadores(detector_pessoas: Detector, frame: np.ndarray) -> List[Detection]:
    df_pessoas = detector_pessoas.predict(frame)
    if not df_pessoas.empty:
        if 'name' in df_pessoas.columns:
            df_pessoas = df_pessoas[df_pessoas["name"] == "person"]
        elif 'class' in df_pessoas.columns:
            df_pessoas = df_pessoas[df_pessoas["class"] == 0]
    return Converter.DataFrame_to_Detections(df_pessoas[df_pessoas["confidence"] > 0.4])

def criar_mascara(frame: np.ndarray, deteccoes: List[Detection]) -> np.ndarray:
    mascara = np.ones(frame.shape[:2], dtype=frame.dtype)
    for det in deteccoes:
        pontos = det.points.astype(int)
        cv2.rectangle(mascara, tuple(pontos[0]), tuple(pontos[1]), 0, -1)
    return mascara

def atualizar_estimador_movimento(estimador_movimento: MotionEstimator, deteccoes: List[Detection], frame: np.ndarray):
    mascara = criar_mascara(frame=frame, deteccoes=deteccoes)
    return estimador_movimento.update(frame, mask=mascara)

def obter_bola_principal(deteccoes: List[Detection], partida: Partida = None) -> Bola:
    bola = Bola(detection=None)
    if partida:
        bola.definir_cor(partida)
    if deteccoes:
        bola.detection = max(deteccoes, key=lambda d: d.data.get("p", 0))
    return bola

## 4. Script Principal de Análise

Esta é a célula principal que orquestra todo o *pipeline* de análise de vídeo. Ela carrega o vídeo, inicializa os detetores, classificadores e rastreadores, e processa cada *frame* para gerar as visualizações táticas.

In [ ]:
import argparse
import cv2
import numpy as np
import PIL
from norfair import Tracker, Video
from norfair.camera_motion import MotionEstimator
from norfair.distances import mean_euclidean
import os

# Criar a pasta 'videos' se não existir
if not os.path.exists('videos'):
    os.makedirs('videos')
    print("Pasta 'videos' criada. Por favor, adicione o seu vídeo (ex: Miami_X_Palmeiras.mp4) a esta pasta.")

# --- Simulação de Argumentos para o Notebook ---
# Em vez de usar argparse, definimos os parâmetros diretamente.
class Args:
    def __init__(self):
        self.video = "videos/Miami_X_Palmeiras.mp4" # Coloque o caminho para o seu vídeo aqui
        self.tatico = True # Mude para False para desabilitar as visualizações
        
        # Forçar todas as flags visuais se --tatico for usado
        self.linhas_formacao = self.tatico
        self.poligonos_formacao = self.tatico
        self.rastro_bola = self.tatico
        self.posse = self.tatico
        
args = Args()

# --- Início do Script de Análise ---
print("⚽ INICIANDO ANÁLISE TÁTICA DE FUTEBOL ⚽")

if not os.path.exists(args.video):
    print(f"\nERRO: Vídeo não encontrado em '{args.video}'. Verifique o caminho e o nome do ficheiro.")
else:
    video = Video(input_path=args.video)
    fps = video.video_capture.get(cv2.CAP_PROP_FPS)

    detector_jogadores = Detector("yolov8x.pt")
    detector_bola = Detector("yolov8x.pt")
    classificador_hsv = HSVClassifier(filters=filtros)
    classificador_final = InertiaClassifier(classifier=classificador_hsv, inertia=30)

    time_casa = Time(nome="Inter Miami", abreviacao="MIA", cor=(221, 160, 221))
    time_visitante = Time(nome="Palmeiras", abreviacao="PAL", cor=(245, 245, 245))
    partida = Partida(time_casa=time_casa, time_visitante=time_visitante, fps=fps)

    rastreador_jogadores = Tracker(distance_function=mean_euclidean, distance_threshold=200)
    rastreador_bola = Tracker(distance_function=mean_euclidean, distance_threshold=250)
    estimador_movimento = MotionEstimator()

    print("\n🚀 PROCESSANDO VÍDEO...")
    total_frames = int(video.video_capture.get(cv2.CAP_PROP_FRAME_COUNT))

    for i, frame in enumerate(video):
        deteccoes_jogadores_raw = obter_deteccoes_jogadores(detector_jogadores, frame)
        deteccoes_bola_raw = obter_deteccoes_bola(detector_bola, frame, usar_bola_esportiva=True)
        
        transformacoes = atualizar_estimador_movimento(estimador_movimento, deteccoes_jogadores_raw + deteccoes_bola_raw, frame)
        
        objetos_rastreados_jogadores = rastreador_jogadores.update(detections=deteccoes_jogadores_raw, coord_transformations=transformacoes)
        objetos_rastreados_bola = rastreador_bola.update(detections=deteccoes_bola_raw, coord_transformations=transformacoes)

        deteccoes_rastreadas = Converter.TrackedObjects_to_Detections(objetos_rastreados_jogadores)
        deteccoes_classificadas = classificador_final.predict_from_detections(detections=deteccoes_rastreadas, img=frame)
        jogadores = Jogador.criar_lista_de_deteccoes(deteccoes_classificadas, partida.times)
        
        bola_deteccoes = Converter.TrackedObjects_to_Detections(objetos_rastreados_bola)
        bola_principal = obter_bola_principal(bola_deteccoes, partida)
        
        partida.atualizar(jogadores, bola_principal)

        frame_pil = PIL.Image.fromarray(frame)
        for jogador in jogadores:
            frame_pil = jogador.desenhar(frame_pil)
        frame_pil = partida.desenhar_elementos(frame_pil, jogadores, args)
        
        frame = np.array(frame_pil)
        video.write(frame)
        
        if i % 30 == 0:
            progresso = (i / total_frames) * 100 if total_frames > 0 else 0
            print(f"⏳ Processando: {progresso:.1f}%", end="\r")

    print("\n" + "="*60 + "\n✅ PROCESSAMENTO CONCLUÍDO!\n" + f"🎬 Vídeo de saída: {video.output_path}\n" + "="*60)

## 5. Conclusão

Este projeto cumpre com sucesso todos os requisitos obrigatórios e desejáveis, entregando uma ferramenta poderosa para a análise tática de futebol. A solução é robusta, bem organizada e atende diretamente à necessidade de um usuário que busca compreender visualmente as dinâmicas de uma partida, com clareza e credibilidade. A utilização de técnicas interpretáveis como o classificador HSV, aliada a modelos de ponta como o YOLOv8, demonstra uma abordagem equilibrada e alinhada aos princípios de uma IA Confiável.